# OTX API DEMO (MMPretrain Example)

## Customization Training API

Select a framework & import adapter modules.

We'll choose MMpretrain here.

"""
Environment:
- mmpretrain-1.0.0rc8
- mmcv-2.0.1
- mmengine-0.7.4
- mmdeploy-1.2.0

"""

## Prepare Dataset & DataLoader
1. Prepare a dataset and enter path into Dataset

    - Convert to OTX's DatasetEntity and Label Schema by leveraging Datumaro's features through paths (path -> Datumaro -> OTX DatasetEntity & LabelSchema)

In [1]:
from otx.v2.adapters.torch.mmengine.mmpretrain import Dataset
dataset = Dataset(
    train_data_roots="../dataset/cifar10-small/train_data",
    val_data_roots="../dataset/cifar10-small/val_data",
    test_data_roots="../dataset/cifar10-small/val_data",
)

/home/harimkan/workspace/repo/otx-fork-3/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/harimkan/workspace/repo/otx-fork-3/venv/lib/python3.9/site-packages/openvino/pyopenvino/__init__.py:10: FutureWarning: The module is private and following namespace `pyopenvino` will be removed in the future
  warnings.warn(message="The module is private and following namespace " "`pyopenvino` will be removed in the future", category=FutureWarning)


2-1. Build Torch Dataset from MMCV config (filepath or dict) -> torch.utils.data.Dataset

    - User can build a dataset from a config file or dictionary.

In [2]:
train_dataloader = dataset.train_dataloader()

print(f"Dataset type: {type(train_dataloader)}")
print(f"Length of DataLoader: {len(train_dataloader)}")
print(f"Dataset size: {len(train_dataloader.dataset)}")
print(f"Number of classes: {dataset.num_classes}")

[*] Detected dataset format: imagenet
[*] Detected task type: CLASSIFICATION
2023-08-04 09:37:41,681 | INFO : Try to create a 0 size memory pool.
Dataset type: <class 'torch.utils.data.dataloader.DataLoader'>
Length of DataLoader: 10
Dataset size: 10
Number of classes: 10


In [3]:
# Customize batch_size & num_workers
train_dataloader = dataset.train_dataloader(
    batch_size=2,
    num_workers=2
)

print(f"DataLoader type: {type(train_dataloader)}")
print(f"Length of DataLoader: {len(train_dataloader)}")
print(f"Dataset size: {len(train_dataloader.dataset)}")

DataLoader type: <class 'torch.utils.data.dataloader.DataLoader'>
Length of DataLoader: 5
Dataset size: 10


In [4]:
val_dataloader = dataset.val_dataloader()

print(f"DataLoader type: {type(val_dataloader)}")
print(f"Length of DataLoader: {len(val_dataloader)}")
print(f"Dataset size: {len(val_dataloader.dataset)}")

DataLoader type: <class 'torch.utils.data.dataloader.DataLoader'>
Length of DataLoader: 10
Dataset size: 10


In [5]:
test_dataloader = dataset.test_dataloader()

print(f"DataLoader type: {type(test_dataloader)}")
print(f"Length of DataLoader: {len(test_dataloader)}")
print(f"Dataset size: {len(test_dataloader.dataset)}")

DataLoader type: <class 'torch.utils.data.dataloader.DataLoader'>
Length of DataLoader: 10
Dataset size: 10


## Prepare Model
### Model provided by OTX

In [6]:
from otx.v2.adapters.torch.mmengine.mmpretrain import list_models
# MMPretrain's model
model_list = list_models("resnet*")
print(model_list)

['resnet101-csra_1xb16_voc07-448px', 'resnet101_8xb16_cifar10', 'resnet101_8xb32_in1k', 'resnet152_8xb16_cifar10', 'resnet152_8xb32_in1k', 'resnet18_8xb16_cifar10', 'resnet18_8xb32_in1k', 'resnet34_8xb16_cifar10', 'resnet34_8xb32_in1k', 'resnet50-arcface_8xb32_inshop', 'resnet50_8xb16_cifar10', 'resnet50_8xb16_cifar100', 'resnet50_8xb256-rsb-a1-600e_in1k', 'resnet50_8xb256-rsb-a2-300e_in1k', 'resnet50_8xb256-rsb-a3-100e_in1k', 'resnet50_8xb32-fp16_in1k', 'resnet50_8xb32_in1k', 'resnet50_8xb8_cub', 'resnet50_barlowtwins-pre_8xb32-linear-coslr-100e_in1k', 'resnet50_byol-pre_8xb512-linear-coslr-90e_in1k', 'resnet50_densecl-pre_8xb32-linear-steplr-100e_in1k', 'resnet50_mocov2-pre_8xb32-linear-steplr-100e_in1k', 'resnet50_mocov3-100e-pre_8xb128-linear-coslr-90e_in1k', 'resnet50_mocov3-300e-pre_8xb128-linear-coslr-90e_in1k', 'resnet50_mocov3-800e-pre_8xb128-linear-coslr-90e_in1k', 'resnet50_simclr-200e-pre_8xb512-linear-coslr-90e_in1k', 'resnet50_simclr-800e-pre_8xb512-linear-coslr-90e_in1k'

In [7]:
# OTX's model
model_list = list_models("otx*")
print(model_list)

['otx_efficientnet_b0', 'otx_mobilenet_v3_large_1']


In [8]:
from otx.v2.adapters.torch.mmengine.mmpretrain import get_model
# mmpretrain's pre-defined model
mmpretrain_model = get_model("resnet18_8xb32_in1k")
print(f"Model type: {type(mmpretrain_model)}")

Model type: <class 'mmpretrain.models.classifiers.image.ImageClassifier'>


In [9]:
## Build Model from model_name

otx_model = get_model(
    model="otx_efficientnet_b0",
    num_classes=dataset.num_classes
)
print(f"Model type: {type(otx_model)}")

2023-08-04 09:37:42,613 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip
2023-08-04 09:37:42,638 | INFO : 'in_channels' config in model.head is updated from -1 to 1280
2023-08-04 09:37:42,704 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip
Model type: <class 'otx.v2.adapters.torch.mmengine.mmpretrain.modules.models.classifiers.custom_image_classifier.CustomImageClassifier'>


In [10]:
## Build Model from Config
otx_model = get_model(
    model="/home/harimkan/workspace/repo/otx-fork-3/src/otx/v2/configs/classification/models/otx_efficientnet_b0.yaml",
    num_classes=dataset.num_classes
)
print(f"Model type: {type(otx_model)}")

2023-08-04 09:37:42,777 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip
2023-08-04 09:37:42,799 | INFO : 'in_channels' config in model.head is updated from -1 to 1280
2023-08-04 09:37:42,868 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip
Model type: <class 'otx.v2.adapters.torch.mmengine.mmpretrain.modules.models.classifiers.custom_image_classifier.CustomImageClassifier'>


## Training

Users can use each framework's training provided by OTX. (Engine)

- The engine requires the necessary models and DataLoaders for each framework.

In [11]:
from otx.v2.adapters.torch.mmengine.mmpretrain.engine import MMPTEngine

# OTX Model Training
engine = MMPTEngine(work_dir="/tmp/otx-test",)

# Training without validation
results = engine.train(
    model=otx_model,
    train_dataloader=train_dataloader,
    max_epochs=3,
)

print(results["checkpoint"])

08/04 09:37:43 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.9.13 (main, Aug 25 2022, 23:26:10) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 828882717
    GPU 0,1: NVIDIA GeForce RTX 3090
    CUDA_HOME: /usr
    NVCC: Cuda compilation tools, release 11.5, V11.5.119
    GCC: gcc (Ubuntu 9.5.0-1ubuntu1~22.04) 9.5.0
    PyTorch: 1.13.1+cu117
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.7
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-genc

In [12]:
val_score = engine.validate(val_dataloader=val_dataloader)
print(f"Val Metric: {val_score}")

test_score = engine.test(test_dataloader=test_dataloader)
print(f"Test Metric: {test_score}")

2023-08-04 09:37:46,317 | WARNING : The config used in the build isstored as an object in the configurationfile because the object doesn't have it.This can result in a non-reusable configs.py.
08/04 09:37:46 - mmengine - INFO - Epoch(val) [3][10/10]    accuracy/top1: 20.0000  data_time: 0.0005  time: 0.0067
08/04 09:37:46 - mmengine - INFO - The best checkpoint with 20.0000 accuracy/top1 at 3 epoch is saved to best_accuracy_top1_epoch_3.pth.
Val Metric: {'accuracy/top1': 20.0}
2023-08-04 09:37:46,747 | WARNING : The config used in the build isstored as an object in the configurationfile because the object doesn't have it.This can result in a non-reusable configs.py.
08/04 09:37:46 - mmengine - INFO - Epoch(test) [10/10]    accuracy/top1: 20.0000  data_time: 0.0005  time: 0.0064
Test Metric: {'accuracy/top1': 20.0}


In [13]:
# OR
# Training with validation
results = engine.train(
    model=otx_model,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    max_epochs=3,
)

print(results["checkpoint"])

08/04 09:37:47 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.9.13 (main, Aug 25 2022, 23:26:10) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 945853104
    GPU 0,1: NVIDIA GeForce RTX 3090
    CUDA_HOME: /usr
    NVCC: Cuda compilation tools, release 11.5, V11.5.119
    GCC: gcc (Ubuntu 9.5.0-1ubuntu1~22.04) 9.5.0
    PyTorch: 1.13.1+cu117
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.7
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-genc

In [14]:
val_score = engine.validate()
print(f"Val Metric: {val_score}")

test_score = engine.test(test_dataloader=test_dataloader)
print(f"Test Metric: {test_score}")

2023-08-04 09:37:48,937 | WARNING : The config used in the build isstored as an object in the configurationfile because the object doesn't have it.This can result in a non-reusable configs.py.
08/04 09:37:49 - mmengine - INFO - Epoch(val) [3][10/10]    accuracy/top1: 20.0000  data_time: 0.0004  time: 0.0066
Val Metric: {'accuracy/top1': 20.0}
2023-08-04 09:37:49,108 | WARNING : The config used in the build isstored as an object in the configurationfile because the object doesn't have it.This can result in a non-reusable configs.py.
08/04 09:37:49 - mmengine - INFO - Epoch(test) [10/10]    accuracy/top1: 20.0000  data_time: 0.0004  time: 0.0066
Test Metric: {'accuracy/top1': 20.0}


## Predict

In [15]:
sample = "/home/harimkan/workspace/repo/demo/dataset/cifar10-small/train_data/bird/image_0042649.jpg"

predict_output = engine.predict(
    model=results["model"],
    checkpoint=results["checkpoint"],
    img=sample
)

print(predict_output)

Loads checkpoint by local backend from path: /tmp/otx-test/best_accuracy_top1_epoch_3.pth


[{'pred_scores': array([0.05100308, 0.05405842, 0.06625324, 0.11286344, 0.17333312,
       0.1106501 , 0.09116399, 0.09596299, 0.0818723 , 0.16283934],
      dtype=float32), 'pred_label': 4, 'pred_score': 0.17333312332630157}]


## Export (with mmdeploy)

In [16]:
export_output = engine.export(
    model=results["model"],
    checkpoint=results["checkpoint"],
)

print(export_output)

08/04 09:37:50 - mmengine - WARNING - Failed to search registry with scope "mmengine" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmengine" is a correct scope, or whether the registry is initialized.
08/04 09:37:50 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
08/04 09:37:50 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "mmpretrain_tasks" registry tree. As a workaround, the current "mmpretrain_tasks" registry in "mmdeploy" is used to build instance. This may cause unexpected failu

/home/harimkan/workspace/repo/otx-fork-3/venv/lib/python3.9/site-packages/mmpretrain/utils/setup_env.py:34: UserWarning: The current default scope "mmengine" is not "mmpretrain", `register_all_modules` will force the current default scope to be "mmpretrain". If this is not expected, please set `init_default_scope=False`.
  warnings.warn(


08/04 09:37:52 - mmengine - INFO - Args for Model Optimizer: mo --input_model="/tmp/otx-test/openvino.onnx" --output_dir="/tmp/otx-test/" --output="output" --input="input" --input_shape="[1, 3, 224, 224]" --mean_values="[123.675, 116.28, 103.53]" --scale_values="[58.395, 57.12, 57.375]" 
08/04 09:37:53 - mmengine - INFO - Check for a new version of Intel(R) Distribution of OpenVINO(TM) toolkit here https://software.intel.com/content/www/us/en/develop/tools/openvino-toolkit/download.html?cid=other&source=prod&campid=ww_2023_bu_IOTG_OpenVINO-2022-3&content=upg_all&medium=organic or on https://github.com/openvinotoolkit/openvino
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.

## Semi-SL Task

In [17]:
# Semi-SL
from otx.v2.adapters.torch.mmengine.mmpretrain import Dataset
semisl_dataset = Dataset(
    train_data_roots="../dataset/cifar10-small/train_data",
    val_data_roots="../dataset/cifar10-small/val_data",
    test_data_roots="../dataset/cifar10-small/val_data",
    unlabeled_data_roots="../dataset/cifar10-small/unlabeled_data"
)

strong_pipeline = [
    dict(type="OTXRandAugment", num_aug=8, magnitude=10),
]

pipeline = {
    "train": [
        dict(type="Resize", scale=[224, 224]),
        dict(type="mmpretrain.PackInputs"),
    ],
    "unlabeled": [
        dict(type="Resize", scale=[224, 224]),
        dict(type="PostAug", keys=dict(img_strong=strong_pipeline)),
        dict(type="mmpretrain.PackMultiKeyInputs", input_key="img", multi_key=["img_strong"]),
    ]
}

semisl_train_dataloader = semisl_dataset.train_dataloader(
    pipeline=pipeline,
    batch_size=2,
    unlabeled_batch_size=8,
)

[*] Detected dataset format: imagenet
[*] Detected task type: CLASSIFICATION
[*] Semisupervised training type detected with unlabeled data: ../dataset/cifar10-small/unlabeled_data
2023-08-04 09:37:53,644 | INFO : possible max iterations = 5


In [18]:
from otx.v2.adapters.torch.mmengine.mmpretrain import get_model
semi_sl_model = dict(
    model=dict(
        type='SemiSLClassifier',
        backbone=dict(
            type='OTXEfficientNet',
            pretrained=True,
            version='b0'),
        neck=dict(
            type='GlobalAveragePooling'),
        head=dict(
            type='SemiLinearClsHead',
            num_classes=10,
            in_channels=1280,
            loss=dict(
                type='CrossEntropyLoss',
                loss_weight=1.0),
            topk=(1, 5)),
        pretrained=None
    )
)


semisl_model = get_model(model=semi_sl_model)

2023-08-04 09:37:53,720 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip


In [19]:
from otx.v2.adapters.torch.mmengine.mmpretrain.engine import MMPTEngine
engine = MMPTEngine(work_dir="/tmp/semi-sl")
results = engine.train(
    model=semisl_model,
    train_dataloader=semisl_train_dataloader,
    max_epochs=3,
)

print(results["checkpoint"])

08/04 09:37:53 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.9.13 (main, Aug 25 2022, 23:26:10) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 256737421
    GPU 0,1: NVIDIA GeForce RTX 3090
    CUDA_HOME: /usr
    NVCC: Cuda compilation tools, release 11.5, V11.5.119
    GCC: gcc (Ubuntu 9.5.0-1ubuntu1~22.04) 9.5.0
    PyTorch: 1.13.1+cu117
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.7
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-genc

## OTX AutoEngine (Automation Training API)
OTX provides a more convenient API called AutoEngine.

- It's more convenient for users to use Engine, which provides auto-configuration and the features provided by OTX without having to choose a framework.
- Prepare Dataset & DataLoader + Prepare Model + OTX Recipes + Training + ETC.
- This will make all of the above steps happen automatically. (Auto: Model Selection & build, Dataset Configuration, Training, etc..)

In [20]:
from otx.v2.api.core.engine import AutoEngine

output_dir = "/tmp/OTX-API-test"
data_roots = "../dataset/cifar10-small/train_data"

engine = AutoEngine(
    work_dir=output_dir,
    train_data_roots=data_roots,
)

# Customization training
results = engine.train(
    batch_size=2,
    max_epochs=2
)
print(results["checkpoint"])

[*] Detected dataset format: imagenet
[*] Detected task type: CLASSIFICATION
2023-08-04 09:37:55,571 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip
2023-08-04 09:37:55,591 | INFO : 'in_channels' config in model.head is updated from -1 to 1280
2023-08-04 09:37:55,659 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip
2023-08-04 09:37:55,661 | WARNING : Currently, OTX does not accept val_dataloader as a dict configuration.
2023-08-04 09:37:55,662 | WARNING : Currently, OTX does not accept test_dataloader as a dict configuration.
2023-08-04 09:37:55,662 | WARNING : In Engine.config, remove ['framework', 'data', 'optimizer', 'max_epochs', 'max_iters', 'val_interval', 'precision'] that are unavailable to the Runner.
08/04 09:37:55 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform

/home/harimkan/workspace/repo/otx-fork-3/venv/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


08/04 09:37:56 - mmengine - INFO - Exp name: otx_train_20230804_093755
08/04 09:37:56 - mmengine - INFO - Epoch(train) [2][5/5]  lr: 5.0000e-03  eta: 0:00:00  time: 0.0291  data_time: 0.0047  memory: 610  loss: 2.5213
08/04 09:37:56 - mmengine - INFO - Saving checkpoint at 2 epochs
/tmp/OTX-API-test/best_accuracy_top1_epoch_2.pth
